# Testing the model

## Install and imports

### 1.1 Mount GCP bucket in Google Colab

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir colab_directory

In [ ]:
!gcsfuse --implicit-dirs foodygs colab_directory

### 1.2 Install Detectron2

In [ ]:
# Requirements

#pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

!pip install -U torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#-f https://download.pytorch.org/whl/torch_stable.html
#torch==1.9.0+cu111
#torchvision==0.10.0+cu111

!pip install cython pyyaml==5.1
!pip install -U pycocotools

#cython

#pyyaml==5.1

#pycocotools

In [ ]:
# This is what is required to run on Google Colab. In the local machine, it is required to install torch and cuda.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# For setup.py

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

### 1.3 Imports

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import os, json, cv2, random
#from google.colab.patches import cv2_imshow  #Only needed for Colab

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# For reading annotations file
from pycocotools.coco import COCO

## 2. Load data

Data is not resized

In [ ]:
import os

# os.chdir allows you to change directories, like cd in the Terminal
os.chdir('/content/colab_directory/foodyai_data')

In [ ]:
# Loading the datasets in coco format and registering them as instances

train_annotations_path = 'Training_2/annotations.json' 
train_images_path = 'Training_2/images'

val_annotations_path = 'Validation_2/annotations.json'
val_images_path = 'Validation_2/images'

train_coco = COCO(train_annotations_path)

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("training_dataset", {},train_annotations_path, train_images_path)
register_coco_instances("validation_dataset", {},val_annotations_path, val_images_path)

## 3. Train model

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
# Check the model zoo and use any of the models ( from detectron2 github repo)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))


cfg.DATASETS.TRAIN = ("training_dataset",)
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2
# Loading pre trained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

### 3.1 Hyperparameters

In [ ]:
# No. of Batchs
cfg.SOLVER.IMS_PER_BATCH = 10   # This is the real "batch size" commonly known to deep learning people

# Learning Rate: 
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR

# No of Interations
cfg.SOLVER.MAX_ITER = 50001 # Try with less to start with

# Images per batch (Batch Size) 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   

# No of Categories(Classes) present
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 323

cfg.OUTPUT_DIR = "logs/"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

## 4. Evaluate model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01 # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 323

cfg.DATASETS.TEST = ("validation_dataset", )
predictor = DefaultPredictor(cfg)

We can also evaluate its performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("validation_dataset", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "validation_dataset")
print(inference_on_dataset(predictor.model, val_loader, evaluator))